In [1]:
"""
DCGAN(Deep convolutional Generative Adbersarial Networks)을 이용한 센서 데이터 augmentation
"""

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import math

In [3]:
##################
# 데이터 load
##################
"""
 Original DataSet
 -- data.shape : (33727, 27) / 'Y': 500개
 --  data_X.shape : (20631,24)
"""
"""
ori_train = pd.read_csv('./data/train_FD0001_true5.csv',encoding= 'cp949')
ori_test = pd.read_csv('./data/test_FD0001_true5.csv',encoding= 'cp949')
#data = pd.concat([ori_train,ori_test], axis = 0)
#data_X = data.iloc[:,:-1].values.tolist()
data = ori_train
data_X = data.iloc[:,2:-1].values.tolist()
data_y = data['label']
data_y = pd.get_dummies(data_y)
data_y
"""
data_X = np.load('X_train_pad.npy')
data_Y = np.load('y_train2.npy')

data_X = np.reshape(data_X, (data_X.shape[0], data_X.shape[1]* data_X.shape[2]))
print(data_X.shape)

data_Y = np.reshape(data_Y, (data_Y.shape[0]))
print(data_Y.shape)

(4818, 272)
(4818,)


In [4]:
##################
# 옵션 설정
##################
total_epoch = 50
batch_size = 512
n_hidden = 64
n_input = data_X.shape[1]
n_noise = 64
learning_rate = 0.01
n_class = len(np.unique(data_Y))
n_multiply = 8 # data 부풀릴 양


In [5]:
data_y = tf.one_hot(data_Y,depth = n_class, dtype=tf.float32)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((data_X, data_y))
dataset = dataset.shuffle(batch_size).repeat().batch(batch_size)

In [7]:
##################
# 신경망 모델 구성
##################
X = tf.placeholder(tf.float32, [None, n_input], name='X')
# 노이즈와 실제 이미지에, 그에 해당하는 숫자에 대한 정보를 넣어주기 위해 사용합니다.
Y = tf.placeholder(tf.float32, [None, n_class], name='Y')
Z = tf.placeholder(tf.float32, [None, n_noise], name='Z')

In [8]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가합니다.
        inputs = tf.concat([noise, labels], 1)
        
        """
        #@@@@@ dcgan
        # TensorFlow 에서 제공하는 유틸리티 함수를 이용해 신경망을 매우 간단하게 구성할 수 있습니다.     
        x = tf.layers.dense(inputs, n_hidden, activation=tf.nn.tanh)
        x = tf.reshape(x, shape=[-1, 1, 1, n_hidden])
        # Deconvolution
        x = tf.layers.conv2d_transpose(x, 16, 4, strides = 2)
        x = tf.layers.batch_normalization(x)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, n_input, 4, strides = 2)
        x = tf.nn.relu(x)
        output = tf.reshape(x, shape = [-1, n_input])
        """
        hidden = tf.layers.dense(inputs, n_hidden, activation = tf.nn.relu)
        hidden = tf.layers.dense(hidden, 32, activation = tf.nn.relu) 
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.tanh)

    return output


def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용하도록 합니다.
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)        
        x = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        x = tf.layers.dense(x, 32, activation=tf.nn.relu)
        output = tf.layers.dense(x, 1, activation=tf.nn.sigmoid)

    return output


def get_noise(batch_size, n_noise):
    return np.random.randn(batch_size, n_noise)

# 생성 모델과 판별 모델에 Y 즉, labels 정보를 추가하여
# labels 정보에 해당하는 이미지를 생성할 수 있도록 유도합니다.
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# 손실함수는 다음을 참고하여 GAN 논문에 나온 방식과는 약간 다르게 작성하였습니다.
# http://bamos.github.io/2016/08/09/deep-completion/
# 진짜 이미지를 판별하는 D_real 값은 1에 가깝도록,
# 가짜 이미지를 판별하는 D_gene 값은 0에 가깝도록 하는 손실 함수입니다.
loss_D_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))

# loss_D_real 과 loss_D_gene 을 더한 뒤 이 값을 최소화 하도록 최적화합니다.
loss_D = loss_D_real + loss_D_gene

# 가짜 이미지를 진짜에 가깝게 만들도록 생성망을 학습시키기 위해, D_gene 을 최대한 1에 가깝도록 만드는 손실함수입니다.
loss_G = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.ones_like(D_gene)))

# TensorFlow 에서 제공하는 유틸리티 함수를 이용해
# discriminator 와 generator scope 에서 사용된 변수들을 쉽게 가져올 수 있습니다.
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='generator')

train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G, var_list=vars_G)

In [9]:
#########
# 신경망 모델 학습
######
generator_saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 편향 초기화
    loss_val_D, loss_val_G = 0, 0

    for epoch in range(total_epoch):
        
        iterator = dataset.make_one_shot_iterator()
        batch_xs, batch_ys = iterator.get_next()
        
        for _ in range(math.ceil(len(data_X)/batch_size)):
            xs, ys = sess.run([batch_xs, batch_ys])
            noise = get_noise(len(ys), n_noise)
            _, loss_val_D = sess.run([train_D, loss_D],
                                     feed_dict={X: xs, Y: ys, Z: noise})
            _, loss_val_G = sess.run([train_G, loss_G],
                                     feed_dict={Y: ys, Z: noise})

        if epoch % 10 == 0:    
            print('Epoch:', '%04d' % epoch,
                  'D loss: {:.4}'.format(loss_val_D),
                  'G loss: {:.4}'.format(loss_val_G))

    generator_saver.save(sess, './model/model.ckpt')
print('최적화 완료!')

Epoch: 0000 D loss: 1.216 G loss: 0.5562
Epoch: 0010 D loss: 1.006 G loss: 0.6931
Epoch: 0020 D loss: 1.006 G loss: 0.6931
Epoch: 0030 D loss: 1.006 G loss: 0.6931
Epoch: 0040 D loss: 1.006 G loss: 0.6931
최적화 완료!


In [10]:
ckpt_state = tf.train.get_checkpoint_state("./model/")
print(type(ckpt_state))

<class 'tensorflow.python.training.checkpoint_state_pb2.CheckpointState'>


In [11]:
print("첫번째 정보 사용법:", ckpt_state.model_checkpoint_path)
print("두번째 정보 사용법:", ckpt_state.all_model_checkpoint_paths)

첫번째 정보 사용법: ./model/model.ckpt
두번째 정보 사용법: ['./model/model.ckpt']


In [12]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories='auto')

###################
# 테스트
###################
"""
ori_test = pd.read_csv('./data/test_FD0001_true5.csv',encoding= 'cp949')
test_X = ori_test.iloc[:,2:-1].values.tolist()

# 꺼지기전 5개일 때
test_y = ori_test.iloc[:,-1]
test_y = test_y.replace('N',0)
test_y = test_y.replace('Y',1)

test_y = ori_test.iloc[:,0]
test_y = pd.get_dummies(test_y)
"""
data_Y = pd.get_dummies(data_Y)

In [13]:
data_N = data_Y[data_Y[1] == 1]

In [14]:
fake_X_N = []

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    sample_size = data_N.shape[0]
    noise = get_noise(sample_size, n_noise)
    for _ in range(n_multiply):    
        samples = sess.run(G, feed_dict={Y: data_N, Z: get_noise(sample_size, n_noise)})
        fake_X_N.append(np.reshape(samples, (samples.shape[0], 16, 17)))
    #samples = sess.run(G, feed_dict={Y: test_y.iloc[0,:], Z: get_noise(sample_size, n_noise)})

In [16]:
fake_X_N = np.asarray(fake_X_N)

In [17]:
fake_X_N.shape

(8, 400, 16, 17)

In [18]:
np.save('fake_X_N', fake_X_N)